# How RMG generates reaction templates?

Han, Kehang (hkh12@mit.edu) 

Nov.10, 2015

## Motivation
The reason I want to make it clear about this functionality is the newly-developed branch `edge_inchi_rxn` produces almost same models as `master` branch does except some divergence which is caused by different kinects for same reactions estimated by RMG. Further investigation shows the different kinetics are stemmed from different reaction templates matched. Therefore, a full understanding of how RMG generates reation templates will efinately help solve the discrepancy between the models from `edhe_inchi_rxn` and those from `master`.

## Set-up
Before investigating, several pre-steps should be set-up.

- import all the necessary modules

- create RMG object and load database needed

- create two reactants for reacting research

In [23]:
from rmgpy.rmg.main import RMG, CoreEdgeReactionModel
from rmgpy.data.rmg import RMGDatabase, database
from rmgpy.data.base import ForbiddenStructureException
from rmgpy.molecule import Molecule
from rmgpy.species import Species
import os

In [24]:
# set-up RMG object
rmg = RMG()
rmg.reactionModel = CoreEdgeReactionModel()

# load kinetic database and forbidden structures
rmg.database = RMGDatabase()
home_path = os.getenv('HOME')
path = os.path.join(home_path, 'Code', 'rmgmaster', 'RMG-database', 'input')
print 'Path is:', path

print("loading forbidden structures...")
rmg.database.loadForbiddenStructures(os.path.join(path, 'forbiddenStructures.py'))

print("loading kinetics families...")
rmg.database.loadKinetics(os.path.join(path, 'kinetics'), \
                          kineticsFamilies=['Disproportionation'])

Path is: /home/kehang/Code/rmgmaster/RMG-database/input
loading forbidden structures...
loading kinetics families...


In [25]:
# create reactants (molecules): C=CC=C and C=CCC
molA = Molecule().fromSMILES("C=CC=C")
molB = Molecule().fromSMILES("C=CCC")
reactants = [molA, molB]
print molA.toAdjacencyList()

1  C u0 p0 c0 {2,S} {3,D} {5,S}
2  C u0 p0 c0 {1,S} {4,D} {6,S}
3  C u0 p0 c0 {1,D} {7,S} {8,S}
4  C u0 p0 c0 {2,D} {9,S} {10,S}
5  H u0 p0 c0 {1,S}
6  H u0 p0 c0 {2,S}
7  H u0 p0 c0 {3,S}
8  H u0 p0 c0 {3,S}
9  H u0 p0 c0 {4,S}
10 H u0 p0 c0 {4,S}



## Make reactions
The reaction-making step into five sub-steps,

- pick the reaction family interested

- match reactants to the family_template's reactants

- create products based on the mapping

- remove duplicated reactions

- generate kinetics template (rate rule) for ther reaction

As you will see, the final kinetics template is actually determined in the second sub-step: the first one of all mappings that lead to same reactions will win because of duplicates removal. 

### Pick reaction family

In [26]:
# pick out the reacting family
families = rmg.database.kinetics.families
disprop_family = families['Disproportionation']

### Map reactants to template reactants

In [27]:
# map reactants to the reacting family's template reactants
family_template = disprop_family.reverseTemplate
mappingsA = disprop_family.\
_KineticsFamily__matchReactantToTemplate(molA, family_template.reactants[1])
mappingsB = disprop_family.\
_KineticsFamily__matchReactantToTemplate(molB, family_template.reactants[0])

At this points, atoms are not labeled, as we can see molA in below.

In [28]:
print molA.toAdjacencyList()

1  C u0 p0 c0 {2,S} {3,D} {5,S}
2  C u0 p0 c0 {1,S} {4,D} {6,S}
3  C u0 p0 c0 {1,D} {7,S} {8,S}
4  C u0 p0 c0 {2,D} {9,S} {10,S}
5  H u0 p0 c0 {1,S}
6  H u0 p0 c0 {2,S}
7  H u0 p0 c0 {3,S}
8  H u0 p0 c0 {3,S}
9  H u0 p0 c0 {4,S}
10 H u0 p0 c0 {4,S}



After using the first mapping for molA, `*2` is assigned to the terminal carbon, which is **from** the carbon losing one `H` atom. This mapping leads molA (`C=CC=C`) to pair with `[CH2]CC=C`.

In [29]:
# assign the labels to atoms based on first mapping
mapA = mappingsA[0]
molA.clearLabeledAtoms()
for atom, templateAtom in mapA.iteritems():
    atom.label = templateAtom.label

print "molA with first mapping: \n" + str(molA.toAdjacencyList())

molA with first mapping: 
1  *2 C u0 p0 c0 {2,S} {3,D} {5,S}
2     C u0 p0 c0 {1,S} {4,D} {6,S}
3  *3 C u0 p0 c0 {1,D} {7,S} {8,S}
4     C u0 p0 c0 {2,D} {9,S} {10,S}
5     H u0 p0 c0 {1,S}
6     H u0 p0 c0 {2,S}
7     H u0 p0 c0 {3,S}
8     H u0 p0 c0 {3,S}
9     H u0 p0 c0 {4,S}
10    H u0 p0 c0 {4,S}



After using the 3rd mapping for molA, `*2` is assigned to the middle carbon, which is **from** the carbon losing one `H` atom. This mapping leads molA (`C=CC=C`) to pair with `C[CH]C=C`.

In [30]:
# assign the labels to atoms based on third mapping
mapA = mappingsA[2]
molA.clearLabeledAtoms()
for atom, templateAtom in mapA.iteritems():
    atom.label = templateAtom.label

print "molA with third mapping: \n" + str(molA.toAdjacencyList())

molA with third mapping: 
1  *3 C u0 p0 c0 {2,S} {3,D} {5,S}
2     C u0 p0 c0 {1,S} {4,D} {6,S}
3  *2 C u0 p0 c0 {1,D} {7,S} {8,S}
4     C u0 p0 c0 {2,D} {9,S} {10,S}
5     H u0 p0 c0 {1,S}
6     H u0 p0 c0 {2,S}
7     H u0 p0 c0 {3,S}
8     H u0 p0 c0 {3,S}
9     H u0 p0 c0 {4,S}
10    H u0 p0 c0 {4,S}



Similar to molB, we have:

In [31]:
# assign the labels to atoms based on first mapping
mapB = mappingsB[0]
molB.clearLabeledAtoms()
for atom, templateAtom in mapB.iteritems():
    atom.label = templateAtom.label
print "molB with first mapping: \n" + str(molB.toAdjacencyList())
# assign the labels to atoms based on third mapping
mapB = mappingsB[2]
molB.clearLabeledAtoms()
for atom, templateAtom in mapB.iteritems():
    atom.label = templateAtom.label
print "molB with third mapping: \n" + str(molB.toAdjacencyList())

molB with first mapping: 
1  *1 C u0 p0 c0 {2,S} {3,S} {5,S} {6,S}
2     C u0 p0 c0 {1,S} {7,S} {8,S} {9,S}
3     C u0 p0 c0 {1,S} {4,D} {10,S}
4     C u0 p0 c0 {3,D} {11,S} {12,S}
5  *4 H u0 p0 c0 {1,S}
6     H u0 p0 c0 {1,S}
7     H u0 p0 c0 {2,S}
8     H u0 p0 c0 {2,S}
9     H u0 p0 c0 {2,S}
10    H u0 p0 c0 {3,S}
11    H u0 p0 c0 {4,S}
12    H u0 p0 c0 {4,S}

molB with third mapping: 
1     C u0 p0 c0 {2,S} {3,S} {5,S} {6,S}
2  *1 C u0 p0 c0 {1,S} {7,S} {8,S} {9,S}
3     C u0 p0 c0 {1,S} {4,D} {10,S}
4     C u0 p0 c0 {3,D} {11,S} {12,S}
5     H u0 p0 c0 {1,S}
6     H u0 p0 c0 {1,S}
7  *4 H u0 p0 c0 {2,S}
8     H u0 p0 c0 {2,S}
9     H u0 p0 c0 {2,S}
10    H u0 p0 c0 {3,S}
11    H u0 p0 c0 {4,S}
12    H u0 p0 c0 {4,S}



### Create product structures

Usually different mapping combinations `(mapA, mapB)` will lead to different products, i.e., different reactions, so which mapping to be applied first doesn't really matter. However, the reaction `#8138` is special enough that different mappings lead to same reaction. In the other words, the very same reaction will have different kinetics templates matched because of different mappings. If reaction duplicates removal is only based on isomorphic checking of reactants and products, which is actually the case, the final kinetics template will be determined by the very first mapping, which may result in non-deterministic behavior if mappings' order varies somehow. 

In [32]:
# create product structures by applying some mapping combination `(mapA, mapB)`
print "mapping combination: 1st mapA and 1st mapB"
for mapA in mappingsA[:1]:
    for mapB in mappingsB[:1]:
        reactantStructures = [molA, molB]
        try:
            productStructures = \
            disprop_family._KineticsFamily__generateProductStructures(reactantStructures, \
                                                                      [mapA, mapB], \
                                                                      False)
        except ForbiddenStructureException:
            pass
        else:
            if productStructures is not None:
                rxn = disprop_family._KineticsFamily__createReaction(reactantStructures, \
                                                                     productStructures, \
                                                                     False)
                if rxn: print rxn

mapping combination: 1st mapA and 1st mapB
<Molecule "C=C[CH]C"> + <Molecule "[CH2]CC=C"> <=> <Molecule "C=CC=C"> + <Molecule "C=CCC">


In [33]:
# create product structures by applying another mapping combination `(mapA, mapB)`
print "mapping combination: 3rd mapA and 3rd mapB"
for mapA in mappingsA[2:3]:
    for mapB in mappingsB[2:3]:
        reactantStructures = [molA, molB]
        try:
            productStructures = \
            disprop_family.\
            _KineticsFamily__generateProductStructures(reactantStructures, \
                                                       [mapA, mapB], \
                                                       False)
        except ForbiddenStructureException:
            pass
        else:
            if productStructures is not None:
                rxn = disprop_family.\
                _KineticsFamily__createReaction(reactantStructures, \
                                                productStructures, False)
                if rxn: print rxn

mapping combination: 3rd mapA and 3rd mapB
<Molecule "[CH2]CC=C"> + <Molecule "C=C[CH]C"> <=> <Molecule "C=CC=C"> + <Molecule "C=CCC">


### Remove duplicates
This step is to only keep the first unique reaction and eliminate other duplicates in the sense of products isomorphism.

But we've already seen, two different mapping combinations `(1st mapA & 1st mapB) vs.(3rd mapA & 3rd mapB)` lead to same product structures, thus same reactions; if you look closer, the pairs of two reactions are actually different. These two mapping combinations stand for different reactant-product pair thus different transition states, probably having very different kinetics. However, current RMG `master` branch chooses to keep only the first occurance which is `(1st mapA & 1st mapB)` instead of `(3rd mapA & 3rd mapB)`, while our `edge_inchi_rxn` branch chooses the opposite.

Therefore, there are two questions in front of us:

1. how is the order of mapping combinations determined in `_KineticsFamily__matchReactantToTemplate`.

2. what is the best way to calculate kinetics if two mapping combinations are both valid.

## New observations

**Observation 1**

- `master` branch maintains same atoms list for molA before and after `_KineticsFamily__matchReactantToTemplate`

- `edge_inchi_rxn` branch will always change atoms list to a final one see below
```
1  C u0 p0 c0 {2,S} {3,D} {5,S}
2  C u0 p0 c0 {1,S} {4,D} {6,S}
3  C u0 p0 c0 {1,D} {7,S} {8,S}
4  C u0 p0 c0 {2,D} {9,S} {10,S}
5  H u0 p0 c0 {1,S}
6  H u0 p0 c0 {2,S}
7  H u0 p0 c0 {3,S}
8  H u0 p0 c0 {3,S}
9  H u0 p0 c0 {4,S}
10 H u0 p0 c0 {4,S}
```

**Observation 2**

- `master` branch gives different order of atoms for molA than `edge_inchi_rxn` does in `Molecule().fromSMILES("C=CC=C")`


If we can solve the two issues above, the `edge_inchi_rxn` should be producing same results as `master` branch does.

## What causes Observation 1
The reason causes change in `edge_inchi_rxn` is
- when creating molA from SMILES, i.e.,`fromSMILES()`, if then falling into `fromOBMol`case, since it currently doesn't call `update()`, the atoms list is un-sorted. A side note is if using `fromRDKitMol`, it does call `update()`, the atoms list will be sorted.
- but after `_KineticsFamily__matchReactantToTemplate` which calls `update()`, the atoms list will change to sorted version.

In [34]:
from rmgpy.molecule.parser import parse_openbabel

mol_test_OB = Molecule()
parse_openbabel(mol_test_OB, "C=CC=C", 'smi')
print "Using openbabel:"
print mol_test_OB.toAdjacencyList()

Using openbabel:
1  C u0 p0 c0 {2,S} {3,D} {5,S}
2  C u0 p0 c0 {1,S} {4,D} {6,S}
3  C u0 p0 c0 {1,D} {7,S} {8,S}
4  C u0 p0 c0 {2,D} {9,S} {10,S}
5  H u0 p0 c0 {1,S}
6  H u0 p0 c0 {2,S}
7  H u0 p0 c0 {3,S}
8  H u0 p0 c0 {3,S}
9  H u0 p0 c0 {4,S}
10 H u0 p0 c0 {4,S}



In [35]:
from rmgpy.molecule.parser import fromRDKitMol
from rdkit import Chem

rdkitmol = Chem.MolFromSmiles("C=CC=C")
mol_test_RD = Molecule()
fromRDKitMol(mol_test_RD, rdkitmol)
print "Using RDKit:"
print mol_test_RD.toAdjacencyList()

Using RDKit:
1  C u0 p0 c0 {2,S} {3,D} {5,S}
2  C u0 p0 c0 {1,S} {4,D} {6,S}
3  C u0 p0 c0 {1,D} {7,S} {8,S}
4  C u0 p0 c0 {2,D} {9,S} {10,S}
5  H u0 p0 c0 {1,S}
6  H u0 p0 c0 {2,S}
7  H u0 p0 c0 {3,S}
8  H u0 p0 c0 {3,S}
9  H u0 p0 c0 {4,S}
10 H u0 p0 c0 {4,S}



## What causes Observation 2
fromSMILES creates Molecule object through several steps below
- `molecule:fromSIMLES` calls `parser:fromSIMLES` calls `__parse` calls `__fromSMILES`
- `__fromSMILES` calls `rdkit.Chem:MolFromSmiles` and `fromRDKitMol`
- `fromRDKitMol`call `molecule:update`
- in `master` branch `molecule:update` calls **`sortAtoms`** while `edge_inchi_rxn` branch it calls **`sortVertices`**

I just checked the latest version of `master` branch has already changed `sortAtoms()` to `sortVertices()` in `update()`, so this observation probably won't be seen again.

## How the order of mapping combinations are determined

In [36]:
molA.clearLabeledAtoms()
templateReactant2 = family_template.reactants[1]
struct = templateReactant2.item
print struct

OR{X_R1, X_R2}


In [37]:
child_structures = struct.getPossibleStructures(disprop_family.groups.entries)# trying to 
c_s = child_structures[0]

In [38]:
print c_s.toAdjacencyList()

1 *2 R!H u0 {2,[D,T]}
2 *3 R!H u0 {1,[D,T]}



In [39]:
print len(disprop_family.groups.entries)

230
